In [47]:
import numpy
import pandas as pd 
import nltk
import string
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [106]:
art_df = pd.read_csv('C:/Users/Powerhouse/Documents/GitHub/Project-High/article-database.csv')
print(art_df.shape)
art_df.head()

(727, 6)


,Unnamed: 0,Name,Url,Text,Tags,Claps
0,0,None,https://medium.com/@elektra.sunn/cheshire-moon...,NaN,[],[]
1,0,None,https://medium.com/@nicolascole77/if-youre-doi...,NaN,[],[]
2,0,The Ultimate Travel Workout and Diet Plans by ...,https://medium.com/@garrettgee/the-bucket-list...,"INTRODUCING THE BUCKET LIST FAMILY Hi, we are...","['Fitness', 'Travel', 'Diet', 'Exercise', 'Vac...",[]
3,0,Building a Responsive Image Gallery,https://medium.com/backticks-tildes/building-a...,In this post you would learn to build a basic...,"['Design', 'Data', 'Growth', 'CSS', 'Web Design']",[]
4,0,When they’re already “forever” for you,https://medium.com/@krisgage/when-theyre-alrea...,Because love that’s “light” can last that lon...,"['Love', 'Relationships', 'Dating', 'Men', 'Wo...",[]


In [107]:
model_db = art_df.drop([art_df.columns[0], art_df.columns[1], art_df.columns[2], art_df.columns[5]], axis=1)
model_db.head()

,Text,Tags
0,NaN,[]
1,NaN,[]
2,"INTRODUCING THE BUCKET LIST FAMILY Hi, we are...","['Fitness', 'Travel', 'Diet', 'Exercise', 'Vac..."
3,In this post you would learn to build a basic...,"['Design', 'Data', 'Growth', 'CSS', 'Web Design']"
4,Because love that’s “light” can last that lon...,"['Love', 'Relationships', 'Dating', 'Men', 'Wo..."


In [108]:
model_db.isnull().sum()

Text    44
Tags     0
dtype: int64

In [109]:
model_db_clean = model_db.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False).reset_index(drop=True)
model_db_clean.head()

,Text,Tags
0,"INTRODUCING THE BUCKET LIST FAMILY Hi, we are...","['Fitness', 'Travel', 'Diet', 'Exercise', 'Vac..."
1,In this post you would learn to build a basic...,"['Design', 'Data', 'Growth', 'CSS', 'Web Design']"
2,Because love that’s “light” can last that lon...,"['Love', 'Relationships', 'Dating', 'Men', 'Wo..."
3,by Jean-Louis Gassée [A delayed Monday Note d...,"['Publishing', 'Apple', 'Apple']"
4,"Today, I was integrating FCM in my iOS app. I...","['Swift', 'iOS', 'Apple']"


In [114]:
# from ast import literal_eval
def func(raw_tags):
    raw_split = raw_tags[1:-1].split(',')
    num_tags = len(raw_split)
    tags_clean = []
    tags_clean.append(raw_split[0][1:-1])
    for i in range(1, num_tags):
       tags_clean.append(raw_split[i][2:-1])
    return tags_clean

model_db_clean['Tags_clean'] = model_db_clean['Tags'].apply(lambda x: func(x))

# model_db_clean.loc[:,'Tags'] = model_db_clean.loc[:,'Tags'].apply(lambda x: literal_eval(x))
print(type(model_db_clean['Tags']))
model_db_clean.head()

<class 'pandas.core.series.Series'>


,Text,Tags,Tags_clean
0,"INTRODUCING THE BUCKET LIST FAMILY Hi, we are...","['Fitness', 'Travel', 'Diet', 'Exercise', 'Vac...","[Fitness, Travel, Diet, Exercise, Vacation]"
1,In this post you would learn to build a basic...,"['Design', 'Data', 'Growth', 'CSS', 'Web Design']","[Design, Data, Growth, CSS, Web Design]"
2,Because love that’s “light” can last that lon...,"['Love', 'Relationships', 'Dating', 'Men', 'Wo...","[Love, Relationships, Dating, Men, Women]"
3,by Jean-Louis Gassée [A delayed Monday Note d...,"['Publishing', 'Apple', 'Apple']","[Publishing, Apple, Apple]"
4,"Today, I was integrating FCM in my iOS app. I...","['Swift', 'iOS', 'Apple']","[Swift, iOS, Apple]"


In [127]:
multi_label_transform = MultiLabelBinarizer()
multi_label_transform.fit(model_db_clean.Tags_clean)

y = multi_label_transform.transform(model_db_clean.Tags_clean)
print(y.shape, model_db_clean.shape)

(683, 500) (683, 3)


In [142]:
cols = ['Text']
for i in list(multi_label_transform.classes_):     
    cols.append(i)

prepd_db = pd.DataFrame()

prepd_db.loc[:, 'Text'] = model_db_clean.loc[:, 'Text']
for i in range(0, y.shape[1]):
        prepd_db.loc[:, i+1] = y[:, i]

prepd_db.columns = cols
prepd_db.drop([''], axis=1, inplace=True)
prepd_db.head()

,Text,2017,AI,API,Abuse,Academia,Accessibility,Activism,Addiction,Adventure,...,World,Writing,Writing Prompts,Writing Tips,Yoga,YouTube,iOS,iOS App Development,iPad,iPhone
0,"INTRODUCING THE BUCKET LIST FAMILY Hi, we are...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,In this post you would learn to build a basic...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Because love that’s “light” can last that lon...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,by Jean-Louis Gassée [A delayed Monday Note d...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Today, I was integrating FCM in my iOS app. I...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
